# Import statements

In [1]:
from pathlib import Path
from helpers import *

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

import pandas as pd

In [2]:
""" ============================================ DO NOT EDIT BELOW ============================================ """
""" environment variable extraction """
try:
    VARS_SET
except NameError:
    VARS_SET = True
    _cwd = %pwd
    _parent_cwd = Path(_cwd).parent
    _start_path = %env PATH
        
%env PATH=/usr/bin:$_start_path

env: PATH=/usr/bin:/home/youn/mambaforge/envs/chronostrain2/bin


# Important global vars

IMPORTANT to edit these!

In [8]:
gene_basedir = Path().resolve().parent.parent / "data" / "chronostrain_seeds" / "Streptococcus_pneumoniae"  # the directory to store the gene files and marker index file.
refseq_metadata_description = "Streptococcus pneumoniae NCTC7465"  # This goes into the metadata of the FASTA records.
mlst_id_prefix = "S_Pneumoniae"


reference_genome_fasta = Path() / "S_Pneumoniae_Reference.fasta"
assert reference_genome_fasta.exists()

# Extract MLST typing genes.

## Streptococcus pneumoniae MLST primers
https://pubmlst.org/organisms/staphylococcus-aureus/primers

The pneumococcal MLST scheme uses internal fragments of the following seven house-keeping genes:

    aroE (shikimate dehydrogenase)
    gdh (glucose-6-phosphate dehydrogenase)
    gki (glucose kinase)
    recP (transketolase)
    spi (signal peptidase I)
    xpt (xanthine phosphoribosyltransferase)
    ddl (D-alanine-D-alanine ligase)

PCR/Sequencing primers

    aroE-up, 5'-GCC TTT GAG GCG ACA GC 
    aroE-dn, 5'-TGC AGT TCA (G/A)AA ACA T(A/T)T TCT AA
    gdh-up, 5'-ATG GAC AAA CCA GC(G/A/T/C) AG(C/T) TT
    gdh-dn, 5'-GCT TGA GGT CCC AT(G/A) CT(G/A/T/C) CC
    gki-up, 5'-GGC ATT GGA ATG GGA TCA CC
    gki-dn, 5'-TCT CCC GCA GCT GAC AC
    recP-up, 5'-GCC AAC TCA GGT CAT CCA GG
    recP-dn, 5'- TGC AAC CGT AGC ATT GTA AC
    spi-up, 5'-TTA TTC CTC CTG ATT CTG TC 
    spi-dn, 5'-GTG ATT GGC CAG AAG CGG AA
    xpt-up, 5'-TTA TTA GAA GAG CGC ATC CT 
    xpt-dn, 5'-AGA TCT GCC TCC TTA AAT AC.
    ddl-up, 5'-TGC (C/T)CA AGT TCC TTA TGT GG 
    ddl-dn, 5'-CAC TGG GT(G/A) AAA CC(A/T) GGC AT 

## Code

In [13]:
!primersearch --version

EMBOSS:6.6.0.0


In [27]:
def perform_primer_search(gene_name: str, forward_primer: str, rev_primer: str, tmp_basedir: Path):
    tmp_dir=tmp_basedir / mlst_id_prefix / gene_name
    tmp_dir.mkdir(exist_ok=True, parents=True)
    return get_primerhit_as_gene(
        chrom_path=reference_genome_fasta,
        cluster_name=gene_name,
        primer1=forward_primer,
        primer2=rev_primer,
        mismatch_pct=10,  # Raised this from 5 to 10, there's some uncertainty in the IUPAC bases.
        tmp_dir=tmp_dir
    )


# ====== Known strain polymorphisms
mlst_genes = []
tmp_basedir = Path() / "__tmp"
mlst_genes.append(perform_primer_search("aroE", "GCCTTTGAGGCGACAGC", "TGCAGTTCAGAAACATATTCTAA", tmp_basedir))  # ambiguous bases removed. (Does EMBOSS handle IUPAC codes?)
mlst_genes.append(perform_primer_search("gdh", "ATGGACAAACCAGCGAGCTT", "GCTTGAGGTCCCATGCTGCC", tmp_basedir))  # ambiguous bases removed. (Does EMBOSS handle IUPAC codes?)
mlst_genes.append(perform_primer_search("gki", "GGCATTGGAATGGGATCACC", "TCTCCCGCAGCTGACAC", tmp_basedir))
# mlst_genes.append(perform_primer_search("recP", "GCCAACTCAGGTCATCCAGG", "TGCAACCGTAGCATTGTAAC", tmp_basedir))  -> Manual FASTA included
# mlst_genes.append(perform_primer_search("spi", "TTATTCCTCCTGATTCTGTC", "GTGATTGGCCAGAAGCGGAA", tmp_basedir))  -> Manual FASTA included
mlst_genes.append(perform_primer_search("xpt", "TTATTAGAAGAGCGCATCCT", "AGATCTGCCTCCTTAAATAC", tmp_basedir))
mlst_genes.append(perform_primer_search("ddl", "TGCCCAAGTTCCTTATGTGG ", "CACTGGGTGAAACCAGGCAT", tmp_basedir))

mlst_genes.append(GeneSequence(name="tkt2", seq=SeqIO.read("S_pneumoniae_transketolase.fasta", format="fasta").seq))  # substitute for recP transketolase
mlst_genes.append(GeneSequence(name="spsB", seq=SeqIO.read("S_pneumoniae_signal_peptidase_I.fasta", format="fasta").seq))  # substitute for spi signal peptidase I

import shutil
shutil.rmtree(tmp_basedir)

In [28]:
# Write each individual MLST gene to fasta.
mlst_index = []
for mlst_gene in mlst_genes:
    record = SeqRecord(
        seq=mlst_gene.seq,
        id=f"{mlst_id_prefix}_{mlst_gene.name}",
        description=f"{refseq_metadata_description} Reference: imputed using MLST primer"
    )

    filename = f'{mlst_gene.name}.fasta'
    with open(gene_basedir / filename, 'wt') as f:
        SeqIO.write([record], f, 'fasta')
    
    mlst_index.append({'Name': f'{mlst_gene.name}', 'Fasta': filename, 'Metadata': 'MLST'})

mlst_index = pd.DataFrame(mlst_index)

In [29]:
mlst_index

,Name,Fasta,Metadata
0,aroE,aroE.fasta,MLST
1,gdh,gdh.fasta,MLST
2,gki,gki.fasta,MLST
3,xpt,xpt.fasta,MLST
4,ddl,ddl.fasta,MLST
5,tkt2,tkt2.fasta,MLST
6,spsB,spsB.fasta,MLST


# Extract MetaPhlAn markers.

In [30]:
taxon_label = 's__Streptococcus_pneumoniae'
metaphlan_version_id = "mpa_vJun23_CHOCOPhlAnSGB_202307"
metaphlan_pkl_path = Path(f"/mnt/e/metaphlan_databases/{metaphlan_version_id}/{metaphlan_version_id}.pkl")

In [31]:
def extract_from_metaphlan(input_metaphlan_pkl: Path):
    parser = MetaphlanParser(input_metaphlan_pkl)

    # Extract reference seqs
    metaphlan_gene_records = []
    for marker_name, record in parser.retrieve_marker_seeds(taxon_label):
        marker_len = len(record.seq)
        print(f"Found marker `{marker_name}` (length {marker_len})")
        metaphlan_gene_records.append(record)
    return metaphlan_gene_records

In [32]:
metaphlan_genes = extract_from_metaphlan(metaphlan_pkl_path)

Searching for marker seeds from MetaPhlAn database: mpa_vJun23_CHOCOPhlAnSGB_202307.
Target # of markers: 82
Found marker `UniRef90_Q8KY50|10__13|SGB8138` (length 550)
Found marker `UniRef90_A0A0B7LEH3|1__5|SGB8138` (length 750)
Found marker `UniRef90_A0A0B7LJ86|1__4|SGB8138` (length 600)
Found marker `UniRef90_Q04HT4|1__4|SGB8138` (length 600)
Found marker `UniRef90_A0A064C162|1__8|SGB8138` (length 1200)
Found marker `UniRef90_Q97Q31|1__5|SGB8138` (length 750)
Found marker `UniRef90_A0A0T7IL18|1__11|SGB8138` (length 1650)
Found marker `UniRef90_Q8DQP1|2__5|SGB8138` (length 550)
Found marker `UniRef90_G0I9W6|1__3|SGB8138` (length 450)
Found marker `UniRef90_A0A0B7MFD2|1__5|SGB8138` (length 750)
Found marker `UniRef90_A0A098ZM76|1__5|SGB8138` (length 700)
Found marker `UniRef90_Q04KL8|2__5|SGB8138` (length 500)
Found marker `UniRef90_Q04KL8|8__11|SGB8138` (length 500)
Found marker `UniRef90_A0A098Z6R5|1__5|SGB8138` (length 750)
Found marker `UniRef90_A0A0B7LW19|3__8|SGB8138` (length 850

In [35]:
# Write each individual MetaPhlAn gene to fasta.
metaphlan_index = []
for metaphlan_gene in metaphlan_genes:
    record = SeqRecord(
        seq=metaphlan_gene.seq,
        id=metaphlan_gene.id,
        description=f"MLST {taxon_label} marker gene version {metaphlan_version_id}"
    )

    uniref_id, middle_tag, sgb_id = metaphlan_gene.id.split("|")
    # Turn this into something posix-friendly.
    filename = f'{uniref_id}-{middle_tag}-{sgb_id}.fasta'
    with open(gene_basedir / filename, 'wt') as f:
        SeqIO.write([record], f, 'fasta')
    
    metaphlan_index.append({'Name': f'{uniref_id}-{middle_tag}-{sgb_id}', 'Fasta': filename, 'Metadata': f'MetaPhlAn {metaphlan_version_id}'})

metaphlan_index = pd.DataFrame(metaphlan_index)

In [36]:
metaphlan_index

,Name,Fasta,Metadata
0,UniRef90_Q8KY50-10__13-SGB8138,UniRef90_Q8KY50-10__13-SGB8138.fasta,MetaPhlAn mpa_vJun23_CHOCOPhlAnSGB_202307
1,UniRef90_A0A0B7LEH3-1__5-SGB8138,UniRef90_A0A0B7LEH3-1__5-SGB8138.fasta,MetaPhlAn mpa_vJun23_CHOCOPhlAnSGB_202307
2,UniRef90_A0A0B7LJ86-1__4-SGB8138,UniRef90_A0A0B7LJ86-1__4-SGB8138.fasta,MetaPhlAn mpa_vJun23_CHOCOPhlAnSGB_202307
3,UniRef90_Q04HT4-1__4-SGB8138,UniRef90_Q04HT4-1__4-SGB8138.fasta,MetaPhlAn mpa_vJun23_CHOCOPhlAnSGB_202307
4,UniRef90_A0A064C162-1__8-SGB8138,UniRef90_A0A064C162-1__8-SGB8138.fasta,MetaPhlAn mpa_vJun23_CHOCOPhlAnSGB_202307
...,...,...,...
77,UniRef90_A0A384ZZY7-1__8-SGB8138,UniRef90_A0A384ZZY7-1__8-SGB8138.fasta,MetaPhlAn mpa_vJun23_CHOCOPhlAnSGB_202307
78,UniRef90_Q8DRF0-1__4-SGB8138,UniRef90_Q8DRF0-1__4-SGB8138.fasta,MetaPhlAn mpa_vJun23_CHOCOPhlAnSGB_202307
79,UniRef90_A0A0E8YZM5-1__7-SGB8138,UniRef90_A0A0E8YZM5-1__7-SGB8138.fasta,MetaPhlAn mpa_vJun23_CHOCOPhlAnSGB_202307
80,UniRef90_D6ZQ15-1__8-SGB8138,UniRef90_D6ZQ15-1__8-SGB8138.fasta,MetaPhlAn mpa_vJun23_CHOCOPhlAnSGB_202307


# Write the index files.

In [37]:
concat_index = pd.concat([mlst_index, metaphlan_index])
concat_index.to_csv(gene_basedir / "marker_seed_index.tsv", sep='\t', index=False, header=False)